In [9]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime, timezone
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize InfluxDB client
client = InfluxDBClient(
    url=os.getenv('INFLUXDB_URL'),
    token=os.getenv('INFLUXDB_TOKEN'),
    org=os.getenv('INFLUXDB_ORG')
)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

# print(f"URL: {os.getenv('INFLUXDB_URL')}")
# print(f"Token: {os.getenv('INFLUXDB_TOKEN')}")
# print(f"Org: {os.getenv('INFLUXDB_ORG')}")
# print(f"Bucket: {os.getenv('INFLUXDB_BUCKET')}")



# Insert New Metric Data Points

In [6]:
def insert_metric(measurement, tags, fields, time=None):
    point = Point(measurement)
    for tag_key, tag_value in tags.items():
        point.tag(tag_key, tag_value)
    for field_key, field_value in fields.items():
        point.field(field_key, field_value)
    # Use timezone-aware datetime in UTC
    point.time(time if time else datetime.now(timezone.utc))
    write_api.write(bucket=os.getenv('INFLUXDB_BUCKET'), org=os.getenv('INFLUXDB_ORG'), record=point)
    # Print success message
    print(f"Inserted point into measurement '{measurement}' with fields: {fields} and tags: {tags}")

# Example Insertion
insert_metric("server_metrics", {"host": "server01"}, {"cpu": 0.85, "memory": 64})
insert_metric("server_metrics", {"host": "server01"}, {"cpu": 0.75, "memory": 70})


Inserted point into measurement 'server_metrics' with fields: {'cpu': 0.85, 'memory': 64} and tags: {'host': 'server01'}
Inserted point into measurement 'server_metrics' with fields: {'cpu': 0.75, 'memory': 70} and tags: {'host': 'server01'}


# Retrieve Metrics Within a Specific Time Range

In [3]:
def retrieve_metrics(query):
    return query_api.query(query=query, org=os.getenv('INFLUXDB_ORG'))

# Example Query (Last 1 hour)
result = retrieve_metrics('from(bucket:"crud") |> range(start: -1h)')
for table in result:
    for record in table.records:
        print(record)


FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2024, 9, 11, 7, 52, 59, 642920, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 9, 11, 8, 52, 59, 642920, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 9, 11, 8, 52, 52, 999914, tzinfo=datetime.timezone.utc), '_value': 0.85, '_field': 'cpu', '_measurement': 'server_metrics', 'host': 'server01'}
FluxRecord() table: 1, {'result': '_result', 'table': 1, '_start': datetime.datetime(2024, 9, 11, 7, 52, 59, 642920, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2024, 9, 11, 8, 52, 59, 642920, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2024, 9, 11, 8, 52, 52, 999914, tzinfo=datetime.timezone.utc), '_value': 64, '_field': 'memory', '_measurement': 'server_metrics', 'host': 'server01'}


# Update Existing Metric Data

In [7]:
# InfluxDB is append-only, so you can't directly update but can add new points.
# Add a new point with an updated value.
insert_metric("server_metrics", {"host": "server01"}, {"cpu": 0.75, "memory": 70})


Inserted point into measurement 'server_metrics' with fields: {'cpu': 0.75, 'memory': 70} and tags: {'host': 'server01'}
